## Data Cleansing

In [39]:
%matplotlib inline
import pandas as pd
import numpy as np
import xgboost as xgb
#from sklearn.preprocessing import LabelEncoder

In [40]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')

## Define Function for feature importance

In [41]:
def get_feature_importance(model):
    Importance = model.get_fscore()
    Importance = list(Importance.items())
    Feature= []
    Score = []
    for each in Importance:
        Feature.append(each[0])
        Score.append(each[1])
    df = pd.DataFrame({'Feature':Feature,'Score':Score}).sort_values(by=['Score'],ascending=[0])
    return df  

## CV Check: With Additional Features

**Add Time Related Features**

In [42]:
# Add month-year
train['timestamp'] = pd.to_datetime(train['timestamp'])
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)
test['timestamp'] = pd.to_datetime(test['timestamp'])
month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)
# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)
week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)
# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek
test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek
# Other feature engineering
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_floor_rev'] = (train['max_floor'] - train['floor']) / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)
test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_floor_rev'] = (test['max_floor'] - test['floor']) / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)
train.apartment_name=train.sub_area.astype(str) + train['metro_km_avto'].astype(str)
test.apartment_name=test.sub_area.astype(str) + train['metro_km_avto'].astype(str)
train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)

**Add Handmade Features**

In [43]:
def get_ratio_preschool(df):
    df['ratio_preschool'] = df['children_preschool'] / (df['children_school'] + 1)
    return df

train = get_ratio_preschool(train)
test = get_ratio_preschool(test)

In [44]:
def get_extra_area(df):
    df['extra_area'] = df['full_sq'] - df['life_sq']
    return df

train = get_extra_area(train)
test = get_extra_area(test)

In [45]:
def get_floor_ratio(df):
    df['floor_ratio'] = df['max_floor'] - df['floor']
    return df

train = get_floor_ratio(train)
test = get_floor_ratio(test)

In [46]:
def get_room_avg_size(df):
    df['room_avg_size'] = (df['life_sq'] - df['kitch_sq']) / (df['num_room'] + 1)
    return df

train = get_room_avg_size(train)
test = get_room_avg_size(test)

# Add average house price!!!

## add moscow average price

In [47]:
def add_moscow_price(df):
    moscow_avg_price = pd.read_csv('moscow_avg_price.csv')

    moscow_avg_price = moscow_avg_price.rename(columns={'timestamp':'t_timestamp'})
    df['date'] = df['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df = df.merge(moscow_avg_price, left_on='date', right_on='t_timestamp').drop(['date', 't_timestamp'], axis=1)
    return df

train_avg_price = add_moscow_price(train)
test_avg_price = add_moscow_price(test)

## add subarea price

In [48]:
def add_subarea_price(df, is_train):
    if is_train:
        train_raw = pd.read_csv('train.csv')
        train_raw = train_raw.rename(columns={'sub_area':'sub_area_raw'})
        df = df.merge(train_raw[['id', 'sub_area_raw']], on='id')
    else:
        test_raw = pd.read_csv('test.csv')
        test_raw = test_raw.rename(columns={'sub_area':'sub_area_raw'})
        df = df.merge(test_raw[['id', 'sub_area_raw']], on='id')
        
    df['date'] = df['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    subarea_avg_price = pd.read_csv('sub_area_avg_price.csv')
    subarea_avg_price = subarea_avg_price.rename(columns={'timestamp':'t_timestamp', 'sub_area':'t_sub_area'})
    
    df = df.merge(subarea_avg_price, how='left', left_on=['date', 'sub_area_raw'], right_on=['t_timestamp', 't_sub_area'])   
    
    return df

train_avg_price = add_subarea_price(train_avg_price, is_train=True).drop(['date', 't_sub_area', 't_timestamp', 'sub_area_raw'], axis=1)
test_avg_price = add_subarea_price(test_avg_price, is_train=False).drop(['date', 't_sub_area', 't_timestamp', 'sub_area_raw'], axis=1)

## Use outside moscow price

In [49]:
def add_outside_moscow_price(df):
    outside_moscow_avg_price = pd.read_csv('outside_avg_price.csv')

    outside_moscow_avg_price = outside_moscow_avg_price.rename(columns={'timestamp':'t_timestamp'})
    df['date'] = df['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df = df.merge(outside_moscow_avg_price, left_on='date', right_on='t_timestamp').drop(['date', 't_timestamp'], axis=1)
    
    return df

train_avg_price = add_outside_moscow_price(train_avg_price)
idx = pd.isnull(train_avg_price['sub_area_avg_price_eur'])
train_avg_price.loc[idx, 'sub_area_avg_price_eur'] = train_avg_price.loc[idx, 'outside_moscow_eur']
train_avg_price.loc[idx, 'sub_area_avg_price_rub'] = train_avg_price.loc[idx, 'outside_moscow_rub']
train_avg_price.loc[idx, 'sub_area_avg_price_usd'] = train_avg_price.loc[idx, 'outside_moscow_usd']
train_avg_price = train_avg_price.drop(['outside_moscow_eur', 'outside_moscow_rub', 'outside_moscow_usd'], axis=1)

test_avg_price = add_outside_moscow_price(test_avg_price)
idx = pd.isnull(train_avg_price['sub_area_avg_price_eur'])
test_avg_price.loc[idx, 'sub_area_avg_price_eur'] = test_avg_price.loc[idx, 'outside_moscow_eur']
test_avg_price.loc[idx, 'sub_area_avg_price_rub'] = test_avg_price.loc[idx, 'outside_moscow_rub']
test_avg_price.loc[idx, 'sub_area_avg_price_usd'] = test_avg_price.loc[idx, 'outside_moscow_usd']
test_avg_price = test_avg_price.drop(['outside_moscow_eur', 'outside_moscow_rub', 'outside_moscow_usd'], axis=1)

In [50]:
train = train_avg_price.drop(['moscow_avg_price_avg_usd', 'moscow_avg_price_high_eur',
       'moscow_avg_price_high_rub', 'moscow_avg_price_high_usd',
       'moscow_avg_price_low_eur', 'moscow_avg_price_low_rub',
       'moscow_avg_price_low_usd', 'sub_area_avg_price_eur','sub_area_avg_price_usd'],axis=1)

In [51]:
test = test_avg_price.drop(['moscow_avg_price_avg_usd', 'moscow_avg_price_high_eur',
       'moscow_avg_price_high_rub', 'moscow_avg_price_high_usd',
       'moscow_avg_price_low_eur', 'moscow_avg_price_low_rub',
       'moscow_avg_price_low_usd', 'sub_area_avg_price_eur','sub_area_avg_price_usd'],axis=1)

## Create New Labels

In [67]:
fill_fullsq = np.mean(train[train.full_sq.notnull()]['full_sq'])

In [68]:
train['full_sq'] = train['full_sq'].fillna(fill_fullsq)
fill_fullsq = np.mean(test[test.full_sq.notnull()]['full_sq'])
test['full_sq'] = test['full_sq'].fillna(fill_fullsq)

In [69]:
train['sub_area_avg_price_rub'] = train['sub_area_avg_price_rub']*train['full_sq']
test['sub_area_avg_price_rub'] = test['sub_area_avg_price_rub']*test['full_sq']

In [70]:
gap = train['price_doc'] - train['sub_area_avg_price_rub']
train['gap'] = gap

In [71]:
col = list(test.columns)[2:]
col = col.remove('sub_area_avg_price_rub')
label = gap

In [77]:
train.price_doc

0         5850000
1         6000000
2         5700000
3        13100000
4        16331452
5         9100000
6         5500000
7         2000000
8         5300000
9         4650000
10        4800000
11        5100000
12        5200000
13        5000000
14        1850000
15        6300000
16        5900000
17        7900000
18        5200000
19        8200000
20        5200000
21        6250000
22        5750000
23        6000000
24        1050000
25        5000000
26        4700000
27        8254400
28        5900000
29        6200000
           ...   
29232    10000000
29233    25039300
29234     4350000
29235     7567425
29236    11438948
29237    10139368
29238     6125400
29239     6373324
29240     6888144
29241     9227657
29242    12610000
29243     2394300
29244     6800000
29245     4066740
29246     6300000
29247     9014972
29248     7800000
29249     6370777
29250     5778893
29251     9500000
29252     5000000
29253    10544070
29254    12000000
29255    10262010
29256     

In [73]:
dtrain = xgb.DMatrix(train[col],train['gap'])

ValueError: cannot label index with a null key

In [111]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)

[0]	train-rmse:14.427+0.00332693	test-rmse:14.427+0.00733507
[50]	train-rmse:1.16127+0.001081	test-rmse:1.16531+0.00683706
[100]	train-rmse:0.317255+0.00347275	test-rmse:0.343961+0.00669282
[150]	train-rmse:0.289677+0.00319547	test-rmse:0.329413+0.00875479
[200]	train-rmse:0.277802+0.0029206	test-rmse:0.327828+0.0086893
[250]	train-rmse:0.268052+0.00344849	test-rmse:0.327477+0.0085827
[300]	train-rmse:0.25905+0.00340065	test-rmse:0.327283+0.0085097


In [124]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse

kf = KFold(n_splits=5,shuffle=False)
col = list(test_avg_price.columns)[2:]
price_doc = train_avg_price['price_doc']
Accuracy = []

for train_index, test_index in kf.split(train_avg_price):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = train_avg_price.iloc[train_index,:], train_avg_price.iloc[test_index,:]
    y_train, y_test = label[train_index], label[test_index]
    dtrain = xgb.DMatrix(X_train[col],y_train)
    dtest = xgb.DMatrix(X_test[col])
    model = xgb.train(params,dtrain,num_boost_round=300)
    print('training done')
    pred = model.predict(dtest)
    #actual_pred = (X_test['Primary Average'] + pred)*X_test['full_sq']
    actual_pred = np.exp(model.predict(dtest))-1
    RMSLE = mse(np.log1p(price_doc[test_index]),np.log1p(actual_pred))**0.5
    print('RMSLE: '+str(RMSLE))
    Accuracy.append(RMSLE)

('TRAIN:', array([ 5853,  5854,  5855, ..., 29259, 29260, 29261]), 'TEST:', array([   0,    1,    2, ..., 5850, 5851, 5852]))
training done
RMSLE: 0.380449141148
('TRAIN:', array([    0,     1,     2, ..., 29259, 29260, 29261]), 'TEST:', array([ 5853,  5854,  5855, ..., 11703, 11704, 11705]))
training done
RMSLE: 0.31098244636
('TRAIN:', array([    0,     1,     2, ..., 29259, 29260, 29261]), 'TEST:', array([11706, 11707, 11708, ..., 17555, 17556, 17557]))
training done
RMSLE: 0.329488971397
('TRAIN:', array([    0,     1,     2, ..., 29259, 29260, 29261]), 'TEST:', array([17558, 17559, 17560, ..., 23407, 23408, 23409]))
training done
RMSLE: 0.321559446221
('TRAIN:', array([    0,     1,     2, ..., 23407, 23408, 23409]), 'TEST:', array([23410, 23411, 23412, ..., 29259, 29260, 29261]))
training done
RMSLE: 0.304411301792
